In [1]:
from galaxyencode import GalaxyEncoder
import os
import tensorflow as tf
from zoobot.data_utils import tfrecord_datasets
from zoobot.estimators import auto_preprocess as preprocess
from zoobot import schemas, label_metadata

question_answer_pairs = label_metadata.decals_pairs
dependencies = label_metadata.get_gz2_and_decals_dependencies(question_answer_pairs)
schema = schemas.Schema(question_answer_pairs, dependencies)

batch_size = 128
train_records_dir = "../auto_shards/train_shards"
eval_records_dir = "../auto_shards/eval_shards"

train_records = [os.path.join(train_records_dir, x) for x in os.listdir(train_records_dir) if x.endswith('.tfrecord')]
eval_records = [os.path.join(eval_records_dir, x) for x in os.listdir(eval_records_dir) if x.endswith('.tfrecord')]

raw_train_dataset = tfrecord_datasets.get_dataset(train_records, schema.label_cols, batch_size, shuffle=True)
raw_test_dataset = tfrecord_datasets.get_dataset(eval_records, schema.label_cols, batch_size, shuffle=False)


preprocess_config = preprocess.PreprocessingConfig(
    label_cols=schema.label_cols,
    input_size=224,
    make_greyscale=False,
    normalise_from_uint8=True
)

train_dataset = preprocess.preprocess_dataset(raw_train_dataset, preprocess_config)
test_dataset = preprocess.preprocess_dataset(raw_test_dataset, preprocess_config)

2021-10-28 10:29:55.886373: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


{smooth-or-featured, indices 0 to 2, asked after None: (0, 2), disk-edge-on, indices 3 to 4, asked after smooth-or-featured_featured-or-disk, index 1: (3, 4), has-spiral-arms, indices 5 to 6, asked after disk-edge-on_no, index 4: (5, 6), bar, indices 7 to 9, asked after disk-edge-on_no, index 4: (7, 9), bulge-size, indices 10 to 14, asked after disk-edge-on_no, index 4: (10, 14), how-rounded, indices 15 to 17, asked after smooth-or-featured_smooth, index 0: (15, 17), edge-on-bulge, indices 18 to 20, asked after disk-edge-on_yes, index 3: (18, 20), spiral-winding, indices 21 to 23, asked after has-spiral-arms_yes, index 5: (21, 23), spiral-arm-count, indices 24 to 29, asked after has-spiral-arms_yes, index 5: (24, 29), merging, indices 30 to 33, asked after None: (30, 33)}


2021-10-28 10:29:57.969527: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-10-28 10:29:58.000375: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-28 10:29:58.001220: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:08.0 name: Tesla P100-PCIE-12GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 11.91GiB deviceMemoryBandwidth: 511.41GiB/s
2021-10-28 10:29:58.001258: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-10-28 10:29:58.060403: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-10-28 10:29:58.060478: I tensorflow/stream_executor/plat

In [2]:
train_dataset

<MapDataset shapes: ((None, 224, 224, 3), (None, 224, 224, 3)), types: (tf.float32, tf.float32)>

In [3]:
model = GalaxyEncoder()
model.compile(optimizer='sgd', loss='mse')
model.encoder.summary()
model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=2,
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir="/idia/projects/hippo/gzd/autoencoder/logs")]
)
#model.save("/idia/projects/hippo/gzd/autoencoder/model_save")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 8)       1160      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 8)         584       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 8)         0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0

2021-10-28 10:30:00.265162: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-10-28 10:30:00.265227: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-10-28 10:30:00.265326: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1611] Profiler found 1 GPUs
2021-10-28 10:30:00.266917: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/.singularity.d/libs
2021-10-28 10:30:00.603621: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcupti.so
2021-10-28 10:30:00.946378: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-10-28 10:30:00.946645: I tensor

Epoch 1/2


2021-10-28 10:30:02.712908: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-10-28 10:30:11.345935: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2021-10-28 10:30:22.550673: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-10-28 10:30:33.053772: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


      1/Unknown - 40s 40s/step - loss: 0.6947

2021-10-28 10:30:40.943259: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-10-28 10:30:40.943313: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.


      2/Unknown - 42s 2s/step - loss: 0.6940 

2021-10-28 10:30:42.980011: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-10-28 10:30:42.980499: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1743] CUPTI activity buffer flushed
2021-10-28 10:30:43.017496: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 3185 callback api events and 2812 activity events. 
2021-10-28 10:30:43.054291: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-10-28 10:30:43.115612: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /idia/projects/hippo/gzd/autoencoder/logs/train/plugins/profile/2021_10_28_10_30_43
2021-10-28 10:30:43.157436: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to /idia/projects/hippo/gzd/autoencoder/logs/train/plugins/profile/2021_10_28_10_30_43/gpu-003.trace.json.gz
2021-10-28 10:30:43.199220: I tensorflow/core/profil

    199/Unknown - 78s 191ms/step - loss: 0.5671

KeyboardInterrupt: 